### Callin Switzer
### Use nn to process sounds



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
import re
from datetime import datetime
import sys
import csv
import seaborn as sns
from collections import Counter
from IPython.display import Image
from scipy import signal

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

In [ ]:
import os

def windowsOrMacDirectories():
    """ Sets base directories for win or mac

       
    """
    if os.environ['COMPUTERNAME'] == 'SHEALMACLEARN':
        DropboxDirect = os.path.join("D:\Dropbox")
#     elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
#         # this excludes your current terminal "/dev/tty"
    elif sys.platform.startswith('darwin'):
        DropboxDirect = os.path.join("/Users/cswitzer/Dropbox")
    else:
        raise EnvironmentError('Unknown computer platform')
    
    baseDir = os.getcwd()
    dataDir = os.path.join(DropboxDirect, 'SonicationBehavior', 'SonBehData')
    figDir = os.path.join(DropboxDirect, 'SonicationBehavior', 'SonBehFigs')
    return baseDir, dataDir, figDir


baseDir, dataDir, figDir = windowsOrMacDirectories()
print(dataDir)

In [ ]:
def readMyFile(filename):
    
    '''Read in csv 10x faster than pandas'''
    
    tmpdta = []
 
    with open(filename, newline="\n") as csvDataFile:
        csvReader = csv.reader(csvDataFile, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
        for row in csvReader:
            tmpdta.append(row)
 
    return(pd.DataFrame(np.transpose(tmpdta)))

In [ ]:
# read in dataset that was pre-classified
buzzClassDataDir = os.path.join("D:\Dropbox\SonicationBehavior\SonBehData\BuzzPartClassification")
buzzClass = pd.read_csv(os.path.join(buzzClassDataDir, 'BuzzClassifications.csv'))
print(buzzClass.shape)
buzzClass.head()

In [ ]:
# read in all data into a single dataframe
bigList = []
freqSpec = []
for ii in range(buzzClass.shape[0]):
    tmp = readMyFile(buzzClass.fileName[ii])
    
    # pad with 0's
    tmp = readMyFile(buzzClass.fileName[ii])
#     pad = np.arange(tmp.iloc[-1,0],tmp.iloc[-1,0]+ 0.02 - np.mean(np.diff(tmp.iloc[:,0])),  np.mean(np.diff(tmp.iloc[:,0])))
#     zx = np.repeat(0, len(pad))
#     pdff = pd.DataFrame( data = {"0":pad, "1":zx} )
#     pdff.columns = tmp.columns

#     tmp = pd.concat([tmp, pdff]).reset_index(drop = True)
    
    # calculate rolling variance
    tmp["varia"] = pd.Series((tmp.iloc[:,1] - np.mean(tmp.iloc[:,1]))).rolling(int(2000), center = True, min_periods = 1).var().tolist()
    
    # calculate frequency spectrum
    f, t, Sxx = signal.spectrogram(tmp.iloc[:,1], 200000, noverlap = 900, nperseg = 1000)
    Sxx = Sxx[0:50, :]
    #scale
    Sxx = Sxx - np.min(Sxx)
    Sxx = Sxx / np.max(Sxx)
    
    
    
    freqSpec.append(pd.DataFrame(np.transpose(Sxx)))
    
    # add classes to data
    tmp["buzz"] = 0
    tmp.loc[buzzClass.buzz1[ii]:buzzClass.buzz2[ii], "buzz"] = 1
    if(np.mod(ii, 10)) == 0:
        print(ii)
    
    tmp["filename"] = buzzClass.fileName[ii]
    bigList.append(tmp)

In [ ]:
plt.plot(pad)

In [ ]:
df = pd.concat(bigList)
df.reset_index(drop = True, inplace = True)

In [ ]:
df.head()

In [ ]:
plt.plot(df.iloc[0:20000, 1])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler((0,1))
mm.fit(df.iloc[:, 1].values.reshape(-1, 1))
df["scaledBuzz"] = mm.transform(df.iloc[:, 1].values.reshape(-1, 1))


subSamp = np.arange(0, 20000, step = 10)# np.arange(0, df.shape[0], step = 5)
plt.figure(figsize=(20,5))
plt.plot(np.array(df.index[subSamp]), np.array(df.iloc[subSamp,5]))
#plt.scatter(df.index[subSamp], df.iloc[subSamp,2]*10, s = 0.5, c = df.iloc[subSamp,3])
plt.show()
df.shape

In [ ]:
print(df.shape)
subSamp = np.arange(0, df.shape[0], step = 10)# np.arange(0, df.shape[0], step = 5)
df = df.iloc[subSamp, :]
print(df.shape)

___
# Finished Data cleaning
___

In [ ]:
# code from : https://gist.github.com/jkleint/1d878d0401b28b281eb75016ed29f2ee


# """
# Example of using Keras to implement a 1D convolutional neural network (CNN) for timeseries prediction.
# """

from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential


def make_timeseries_regressor(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=4):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.
    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).
    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    model = Sequential((
        # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
        # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
        # the input timeseries, the activation of each filter at that position.
        
        #Conv1D(activation="relu", filters=4, kernel_size=5)
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        MaxPooling1D(),     # Downsample the output of convolution by 2X.
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='sigmoid'),     # For binary classification, change the activation to 'sigmoid'
    ))
    #model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    # To perform (binary) classification instead:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model


def make_timeseries_instances(timeseries, seqLabels, window_size):
    """Make input features and prediction targets from a `timeseries` for use in machine learning.
    :return: A tuple of `(X, y, q)`.  `X` are the inputs to a predictor, a 3D ndarray with shape
      ``(timeseries.shape[0] - window_size, window_size, timeseries.shape[1] or 1)``.  For each row of `X`, the
      corresponding row of `y` is the next value in the timeseries.  The `q` or query is the last instance, what you would use
      to predict a hypothetical next (unprovided) value in the `timeseries`.
    :param ndarray timeseries: Either a simple vector, or a matrix of shape ``(timestep, series_num)``, i.e., time is axis 0 (the
      row) and the series is axis 1 (the column).
    :param int window_size: The number of samples to use as input prediction features (also called the lag or lookback).
    """
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0]
    X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
    y = seqLabels
    q = np.atleast_3d([timeseries[-window_size:]])
    return X, y, q


def evaluate_timeseries(timeseries, seqLabels, window_size, epochs=100):
    """Create a 1D CNN regressor to predict the next value in a `timeseries` using the preceding `window_size` elements
    as input features and evaluate its performance.
    :param ndarray timeseries: Timeseries data with time increasing down the rows (the leading dimension/axis).
    :param int window_size: The number of previous timeseries values to use to predict the next.
    """
    filter_length = 5
    nb_filter = 4
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       # Convert 1D vectors to 2D column vectors

    nb_samples, nb_series = timeseries.shape
    print('\n\nTimeseries ({} samples by {} series):\n'.format(nb_samples, nb_series), timeseries)
    model = make_timeseries_regressor(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)
    print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, nb_filter, filter_length))
    model.summary()

    X, y, q = make_timeseries_instances(timeseries, seqLabels, window_size)
    print('\n\nInput features:', X, '\n\nOutput labels:', y, '\n\nQuery vector:', q, sep='\n')
    test_size = int(0.3 * nb_samples)           # In real life you'd want to use 0.2 - 0.5
    train_size = nb_samples - test_size
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-(test_size + window_size)], y[train_size - window_size: -window_size]
    history= model.fit(X_train, y_train, epochs=epochs, batch_size=2000, validation_data=(X_test, y_test))

    pred = model.predict(X_test)
    return(pred, X_train, X_test, y_train, y_test, model, history)


In [ ]:
timeseries = df.iloc[:, 5]
seqLabels = df.iloc[:, 3]

ts_length = df.shape[0]
window_size = 1000

print('\nSimple single timeseries vector prediction')
pred, X_train, X_test, y_train, y_test, model, history = evaluate_timeseries(timeseries,seqLabels, window_size, epochs = 20)


In [ ]:
#train for more epochs
history= model.fit(X_train, y_train, epochs=100, batch_size=2000, validation_data=(X_test, y_test))
pred = model.predict(X_test)

In [ ]:
print(history.history.keys())


plt.plot(history.history['loss'], c = "orange")
plt.title('Neural network loss and accuracy')
plt.ylabel('loss')

plt.xlabel('epoch')




plt.plot(history.history['binary_accuracy'], c = "purple")
plt.legend(['train_loss', 'train_acc'], loc='center')


plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.array(y_test)[0:50000], ((np.array(pred) > 0.5) * 1)[0:50000])

In [ ]:
X_train.shape
y_train.shape

In [ ]:
plt.plot(X_train[0:20000,0, -1])
plt.plot(np.array(y_train)[0:20000] + 1.1, c= 'pink')


In [ ]:

plt.figure(figsize=[20, 10])
plt.plot(X_test[0:50000,0, -1])
plt.plot(np.array(y_test)[0:50000] + 1.1, c= 'pink')
plt.plot(((np.array(pred) > 0.5) * 1)[0:50000] + 1, c= 'green')

In [ ]:
X_test.shape

In [ ]:
df.head()


___

### 1D convolution on spectrogram 

In [ ]:
# add padding to beginning of dataset

df = pd.concat(bigList)
df.reset_index(drop = True, inplace = True)
mm = MinMaxScaler((0,1))
mm.fit(df.iloc[:, 1].values.reshape(-1, 1))
df["scaledBuzz"] = mm.transform(df.iloc[:, 1].values.reshape(-1, 1))

samplingFreq = 200000
df["time"] = df.index / samplingFreq


seq = df.loc[:,"scaledBuzz"]



xx = df.loc[:, "time"]

# add padding
padLength = 100000
s2 = np.hstack([np.zeros(padLength)+np.mean(seq),seq,np.zeros(padLength)+np.mean(seq)])

plt.figure(figsize=[20, 10])
nperS = 3000
f, t, Sxx = signal.spectrogram(np.array(s2), fs =samplingFreq, noverlap = 2500, nperseg = nperS, nfft = 10000)
Sxx = Sxx[0:100, :]
f = f[0:100]



# now resize to fit original data

t = t - (padLength / samplingFreq)
keepTime = (t >= np.min(xx)) & (t <= np.max(xx))
t = t[keepTime]
Sxx = Sxx[:, keepTime]

plt.pcolormesh(t[0:1000], f, Sxx[:, 0:1000])
plt.ylim(0,1000)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.plot(xx[0:500000], (seq[0:500000]*300) + 250, alpha = 0.3, c = 'orange')
plt.show()


In [ ]:
# now that I've got the spectrum fixed, I'll try some other methods, before NN

In [ ]:
from skimage.transform import resize
bb = resize(Sxx, output_shape = [100, df.shape[0]], order = 1, mode = 'reflect')


In [ ]:
plt.figure(figsize = [20, 10])
plt.imshow(bb[:, 0:800000], aspect = "auto", origin = 'lower')
plt.plot((df.loc[0:800000, "scaledBuzz"])*10 -30)
plt.plot((df.loc[0:800000, "varia"])*750 -20)
plt.plot((df.loc[0:800000, "varia"]>0.001)* 20)
plt.plot((df.loc[0:800000, "buzz"])* 20)
#plt.axes(frameon = 0)
plt.show()

In [ ]:
cc = pd.DataFrame(np.transpose(bb))
dd = pd.concat([df,cc], axis = 1 )
dd.shape

In [ ]:
plt.imshow(np.array(cc), aspect = "auto", origin = 'lower')

In [ ]:
# build test/train sets
np.random.seed(1235999)
trainSamps = np.random.choice(np.unique(dd.filename), 
                              size = int(0.75*len(np.unique(dd.filename))),
                             replace = False)



train = dd[dd['filename'].isin(trainSamps)]
train.reset_index(drop = True, inplace = True)

test = dd[~dd['filename'].isin(trainSamps)]
test.reset_index(drop = True, inplace = True)

In [ ]:
train.head()

In [ ]:
Ytrain= np.array(train["buzz"])
Xtrain = np.array(train.drop(["buzz", "filename", "time", "scaledBuzz"], axis = 1).iloc[:, 2:])

Ytest= np.array(test["buzz"])
Xtest = np.array(test.drop(["buzz", "filename", "time", "scaledBuzz"], axis = 1).iloc[:, 2:])


# scale each column -- save scaling factors
dfMin = Xtrain[:,1:].min(axis = 0)
dfMax =  Xtrain[:,1:].max(axis = 0)

varMin = Xtrain[:,0].min()
varMax =  Xtrain[:,0].max()

Xtrain[:,1:] =  Xtrain[:,1:] - dfMin
Xtrain[:,1:] =  Xtrain[:,1:] / dfMax
Xtrain[:,0] =  Xtrain[:,0] - varMin
Xtrain[:,0] =  Xtrain[:,0] / varMax

Xtest[:,1:] =  Xtest[:,1:] - dfMin
Xtest[:,1:] =  Xtest[:,1:] / dfMax
Xtest[:,0] =  Xtest[:,0] - varMin
Xtest[:,0] =  Xtest[:,0] / varMax

plt.imshow(np.array(np.transpose(Xtrain)), aspect='auto')

#plt.plot(Xtrain[:,0]*100, c= 'red', markersize= 1000)

# plt.show()

# plt.plot(train.iloc[:,1])
# plt.plot(Xtrain[:,0]*5, c= 'red', markersize= 1000)

In [ ]:
np.mean(Ytrain)

In [ ]:
stt = time.time()

from sklearn import svm
clf = svm.LinearSVC(dual = False)
clf.fit(Xtrain[:, 0].reshape(-1, 1), Ytrain)  
preds = clf.predict(Xtest[:, 0].reshape(-1, 1))
from sklearn.metrics import accuracy_score
print(accuracy_score(clf.predict(Xtrain[:, 0].reshape(-1, 1)), Ytrain))
print(accuracy_score(preds, Ytest))
print(time.time() - stt)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc

stt = time.time()
forest = rfc(n_jobs = -1, class_weight="balanced")
forest.fit(Xtrain[:,0].reshape(-1, 1), Ytrain)
print(time.time() - stt)

In [ ]:
preds = forest.predict(Xtest.reshape(-1, 1))
from sklearn.metrics import accuracy_score
print(accuracy_score(forest.predict(Xtrain[:,0].reshape(-1, 1)), Ytrain))
accuracy_score(preds, Ytest)


In [ ]:
print(accuracy_score(forest.predict(Xtrain), Ytrain))

In [ ]:
#efsfees

In [ ]:
p2 = (Xtest[:,0] > 0.001)*1
print(accuracy_score(p2, Ytest))

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(p2, Ytest)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(p2, Ytest)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))


In [ ]:
# signal practice
samplingFreq = 200000

xx = np.linspace(0, 1, num = samplingFreq)
seq = np.sin(250 * 2 * np.pi * xx + (30*np.sin(2*np.pi* 3 * xx))) #+ np.sin(800*2*np.pi* xx)

seq[40000:80000] = 0

mm = MinMaxScaler((0,1))
mm.fit(seq.reshape(-1, 1))
s1= mm.transform(seq.reshape(-1, 1))


# add padding
padLength = 100000
s2 = np.hstack([np.zeros(padLength) ,s1.reshape(-1,),np.zeros(padLength) - 10])

plt.figure(figsize=[20, 10])
nperS = 11000
f, t, Sxx = signal.spectrogram(np.array(s2), fs =samplingFreq, noverlap = 3000, nperseg = 3500)
Sxx = Sxx[0:50, :]
f = f[0:50]

plt.pcolormesh(t, f, Sxx)
plt.ylim(0,400)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.plot(xx, (seq*30) + 250)
plt.show()

# now resize to fit original data

t = t - (padLength / samplingFreq)
keepTime = (t >= np.min(xx)) & (t <= np.max(xx))
t = t[keepTime]
Sxx = Sxx[:, keepTime]

plt.pcolormesh(t, f, Sxx)
plt.ylim(0,400)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.plot(xx, (seq*30) + 250)
plt.show()


In [ ]:
# signal practice
samplingFreq = 200000

xx = np.linspace(0, 1, num = samplingFreq)
seq = np.sin(100 * 2 * np.pi * xx + (10*np.sin(2*np.pi* 3 * xx))) #+ np.sin(800*2*np.pi* xx)

seq[40000:80000] = 0

from scipy import signal
plt.figure(figsize=[20, 10])
nperS = 16000
f, t, Sxx = signal.spectrogram(np.array(seq), 200000, noverlap = 1550, nperseg = nperS)
Sxx = Sxx[0:90,:]
f = f[0:90]
plt.pcolormesh(t, f, Sxx)
plt.ylim(0,1000)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')





plt.plot(xx , (seq*100))


# now resize to fit original data
plt.plot(t, Sxx[15, :]*10000000)
print(np.min(t), np.max(t))


In [ ]:
pd.DataFrame(np.transpose([t, Sxx[20, :]*100]))
min(t), max(t)

In [ ]:
FS = pd.concat(freqSpec)
plt.imshow((FS.transpose()).iloc[:,  0:500, ], aspect = "auto", origin = 'lower')

In [ ]:
subSamp = np.linspace(0, df.shape[0]-1, num = t.shape[0]).astype(int)

In [ ]:
df2 = df.iloc[subSamp, :]

In [ ]:
plt.plot(x2[x2 < 2.5], y2[x2 < 2.5]*300)

x2 = (np.arange() / 20000)
y2 = df2.loc[:,"scaledBuzz"]
plt.plot(x2[x2 < 2.5], y2[x2 < 2.5])